# Bootstrap

The bootstrap is an alternative to the NHST framework already discussed. The primary benefit of the bootstrap is that it comes with fewer assumptions then the NHST framework. The only real assumption when doing a bootstrap approach is that the sample is obtained randomly from the population, an assumption already made in the NHST framework. The primary drawback of the bootstrap approach is that it is computationally expensive, therefore, it can take time to peform the procedure. 

## Bootstrapping Steps

The following are the steps when performing a bootstrap. 

1. Treat the sample data as the population.
2. Resample, with replacement, from the sample data, ensuring the new sample is the same size as the original. 
3. Estimate the model using the resampled data from step 2. 
4. Repeat steps 2 and 3 many many times (eg, 10,000 or more). 
5. Visualize distribution of effect of interest

## Resampling with replacement

What is meant by sampling with replacement? Let's do an example.

In [ ]:
library(tidyverse)

fruit <- data.frame(name = c('watermelon', 'apple', 'orange', 'kumquat', 'grapes', 'canteloupe', 'kiwi', 'banana')) %>%
    mutate(obs_num = 1:n())

slice_sample(fruit, n = nrow(fruit), replace = FALSE)

In [ ]:
slice_sample(fruit, n = nrow(fruit), replace = TRUE)

## More practical example

Let's load some data to do a more practical example. The following data come from a [Tidy Tuesday example](https://github.com/rfordatascience/tidytuesday/blob/master/data/2020/2020-07-14/readme.md) on astronauts and their missions in space. 

| variable                 | class     | description                                               |
| :----------------------- | :-------- | --------------------------------------------------------- |
| id                       | double    | ID                                                        |
| number                   | double    | Number                                                    |
| nationwide_number        | double    | Number within country                                     |
| name                     | character | Full  name                                                |
| original_name            | character | Name in original language                                 |
| sex                      | character | Sex                                                       |
| year_of_birth            | double    | Year of birth                                             |
| nationality              | character | Nationality                                               |
| military_civilian        | character | Military status                                           |
| selection                | character | Name of selection program                                 |
| year_of_selection        | double    | Year of selection program                                 |
| mission_number           | double    | Mission number                                            |
| total_number_of_missions | double    | Total number of missions                                  |
| occupation               | character | Occupation                                                |
| year_of_mission          | double    | Mission year                                              |
| mission_title            | character | Mission title                                             |
| ascend_shuttle           | character | Name of ascent shuttle                                    |
| in_orbit                 | character | Name of spacecraft used in orbit                          |
| descend_shuttle          | character | Name of descent shuttle                                   |
| hours_mission            | double    | Duration of mission in hours                              |
| total_hrs_sum            | double    | Total duration of all missions in hours                   |
| field21                  | double    | Instances of EVA by mission                               |
| eva_hrs_mission          | double    | Duration of extravehicular activities  during the mission |
| total_eva_hrs            | double    | Total duration of all extravehicular activities in hours  |

In [ ]:
astronauts <- readr::read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-07-14/astronauts.csv') %>%
   mutate(year_centered = year_of_mission - min(year_of_mission))

head(astronauts)

In [ ]:
library(ggformula)

theme_set(theme_bw(base_size = 18))

gf_point(hours_mission ~ year_of_mission, data = astronauts, size = 5) %>%
  gf_smooth(method = 'loess', size = 2) %>%
  gf_smooth(method = 'lm', linetype = 2, color = 'lightblue', size = 2) %>%
  gf_labs(x = 'Year of Mission',
          y = "Mission Hours")

In [ ]:
mission_lm <- lm(hours_mission ~ year_of_mission, data = astronauts)

summary(mission_lm)
confint(mission_lm)

In [ ]:
mission_lm <- lm(hours_mission ~ year_centered, data = astronauts)

summary(mission_lm)
confint(mission_lm)

## Resample these data

For the astronauts data, to resample, a similar idea can be made. Essentially, we are treating these data as a random sample of some population of space missions. We again, would resample, with replacement, which means that multiple missions would likely show up in the resampling procedure. 

In [ ]:
slice_sample(astronauts, n = nrow(astronauts), replace = TRUE) %>% 
  count(id) %>%
  gf_col(n ~ id)

In [ ]:
resamp_lm <- slice_sample(astronauts, n = nrow(astronauts), replace = TRUE) %>% 
  lm(hours_mission ~ year_centered, data = .)

summary(resamp_lm)
confint(resamp_lm)

Let's now repeat this a bunch of times. 

In [ ]:
set.seed(2021)

resample_astronaut <- function(...) {
  astronaut_resample <- slice_sample(astronauts, n = nrow(astronauts), replace = TRUE)

  astronaut_resample %>%
    lm(hours_mission ~ year_centered, data = .) %>%
    coef(.) %>%
    broom::tidy()
}

In [ ]:
resample_astronaut(astronauts)

In [ ]:
astronaut_coef <- map_dfr(1:10000, resample_astronaut)

gf_density(~ x, data = astronaut_coef) %>%
  gf_facet_wrap(~ names, scales = 'free') %>%
  gf_labs(x = "Regression Estimates")